In [5]:
import json
from datetime import datetime

import pandas as pd
import requests
from sqlalchemy import select
from sqlalchemy.orm import aliased

from database import Database
from utils import DistributorApi

In [6]:
api = DistributorApi()
db = Database()

In [7]:
rules = api.get_stopsale_rules()
df_rules = pd.json_normalize(rules, sep="_")
df_rules = df_rules[
    [
        "id",
        "name",
        "description",
        "tag",
        "edit_state",
        "obsolete",
        "updated_by",
        "updated_on",
        "credential_level",
        "credential_list",
        "provider_level",
        "provider_list",
        "hotel_level",
        "hotel_list",
        "destination_level",
        "destination_list",
        "rate",
        "refundable",
        "market_level",
        "market_list",
        "meal_level",
        "meal_list",
        "check_in_level",
        "check_in_from",
        "check_in_to",
        "booking_date_level",
        "booking_date_from",
        "booking_date_to",
        "range_level",
        "range_from",
        "range_to",
        "max_release",
        "days_of_week_level",
        "days_of_week_list",
        "age",
        "room_level",
        "room_list",
        "num_of_nights_level",
        "num_of_nights_list",
        "hours_level",
        "hours_list",
        "dynamic_commission",
    ]
]

Rules retrieved successfully.


KeyError: "None of [Index(['id', 'name', 'description', 'tag', 'edit_state', 'obsolete',\n       'updated_by', 'updated_on', 'credential_level', 'credential_list',\n       'provider_level', 'provider_list', 'hotel_level', 'hotel_list',\n       'destination_level', 'destination_list', 'rate', 'refundable',\n       'market_level', 'market_list', 'meal_level', 'meal_list',\n       'check_in_level', 'check_in_from', 'check_in_to', 'booking_date_level',\n       'booking_date_from', 'booking_date_to', 'range_level', 'range_from',\n       'range_to', 'max_release', 'days_of_week_level', 'days_of_week_list',\n       'age', 'room_level', 'room_list', 'num_of_nights_level',\n       'num_of_nights_list', 'hours_level', 'hours_list',\n       'dynamic_commission'],\n      dtype='object')] are in the [columns]"

In [ ]:
df_rules.shape

In [ ]:
# clients_credential from discovery

cc = aliased(db.clients_credential)
df_credentials = pd.read_sql(
    sql=select(cc.c.id.label("credential_id"), cc.c.enabled), con=db.engine.connect()
)

In [ ]:
# fitler for tags empty, product, customer request and credential list

df_rules_filtered = df_rules[
    (df_rules["credential_level"] == 1) & (df_rules["tag"].isin([-1, 1, 3]))
]
df_rules_expanded = df_rules_filtered.explode("credential_list").reset_index(drop=True)

In [ ]:
df_rules_filtered.shape

In [ ]:
df_rules_expanded["credential_id"] = df_rules_expanded["credential_list"]
df_rules_expanded = df_rules_expanded.merge(
    df_credentials, on="credential_id", how="left"
)

In [ ]:
df_rules_expanded.shape

In [4]:
df_regrouped = (
    df_rules_expanded.groupby("id")
    .agg(
        credential_count=("credential_id", "count"),
        enabled=("enabled", lambda x: (x == True).sum()),
        disabled=("enabled", lambda x: (x == False).sum()),
    )
    .reset_index()
)

df_regrouped["disabled_percentage"] = (df_regrouped["disabled"] * 100.0) / df_regrouped[
    "credential_count"
]
df_regrouped = df_regrouped[df_regrouped["disabled_percentage"] == 100]

NameError: name 'df_rules_expanded' is not defined

In [11]:
df_rules_inactive_creds = df_rules_filtered.merge(df_regrouped, on="id", how="right")

In [12]:
df_rules_inactive_creds.shape

(2, 46)

In [13]:
rule_ids = list(df_regrouped["id"])
rule_ids

['394825', '395101']

In [16]:
# Uncomment to delete
# deleted_rules = api.delete_rules(rule_ids)

Rules deleted successfully.


In [17]:
# df_rules_inactive_creds.to_csv(
#     f"deleted_rules_{datetime.today().strftime('%Y%m%d_%H%M%S')}_with_inactive_creds.csv",
#     index=False,
# )